# Strava segment data

Choose some activity ID or the LAST one if `LAST=True`

In [1]:
activity=4088494919
date='September 20, 2020'
LAST=True

In [2]:
import helium
import pandas as pd
import os
import time
import sys
def metadata(**kwargs):
    return kwargs
db_dir='db'
db_file='{}/strava.json'.format(db_dir)

In [3]:
bkp=pd.read_json(db_file)
if not LAST and not bkp[bkp.Info.astype(str).str.contains(date)].empty:
    sys.exit('Check if already exists in bkp')

In [4]:
os.makedirs(db_dir,exist_ok=True)

if os.path.isfile(db_file):
    df=pd.read_json(db_file)
else:
    df=pd.DataFrame()

browser=helium.start_chrome("https://www.strava.com/login")

helium.write('diego.restrepo@gmail.com',into='Your Email')

helium.write('opoqdh11',into='Password')

helium.click('Log In')

time.sleep(10)
browser.maximize_window()

if LAST:
    #Required to activate card-section below
    lt=[t for t in browser.find_elements_by_class_name('card-section') 
       if t.text.find('Latest Activity')>-1][0]

    lt.find_element_by_tag_name('a').click()

else:
    browser.get('https://www.strava.com/activities/{}'.format(activity))
# Web page with latest activity

# General info

time.sleep(10)
dt=browser.find_element_by_class_name('details')

date=dt.find_element_by_tag_name('time').text

name=dt.find_element_by_tag_name('h1').text

kk=browser.find_elements_by_tag_name("ul")

q=[ k for k in kk if k.text.find('\nDistance')>-1][0]

distance,time,elevation=(t.text for t in q.find_elements_by_tag_name('strong') )

q=[ k for k in kk if k.text.find('\nEstimated Avg Power')>-1][0]

power,energy=(t.text for t in q.find_elements_by_tag_name('strong') )

# Load tables

HIDDEN=True
try:
    browser.find_element_by_id('show-hidden-efforts').click()
except:
    HIDDEN=False
    pass

html=browser.page_source

ll=pd.read_html(html)

# General info table

mtdt=ll[0].groupby('Unnamed: 0').sum().transpose().to_dict()

mtdt.update(  metadata(date=date,name=name,distance=distance,time=time,
         elevation=elevation,power=elevation,energy=energy)
   )

#Fix time
import time

# Segments

sgmt=ll[1][['Name','Time','Speed','Power']].reset_index(drop=True)

if HIDDEN:
    sgmt=sgmt.append( ll[2].rename({2:'Name',4:'Time',5:'Speed',6:'Power'},
            axis='columns')[['Name','Time','Speed','Power']] 
           ).reset_index(drop=True)

#Add general info
sgmt['Info']=''

sgmt['Info'] = [ mtdt  for i in sgmt.index]

In [5]:
sgmt

,Name,Time,Speed,Power,Info
0,Suramerica - Caldas 7.92km 137m 1.7%,26:55,17.7km/h,197W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
1,unicornio ancon 1.13km 63m 5.4%,5:18,12.9km/h,169W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
2,Sprint (El ciclista) 0.34km 17m 5.1%,2:03,10.0km/h,133W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
3,El Ciclista - Minas 21.58km 781m 3.6%,2:01:52,10.6km/h,179W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
4,Alto de Gazu - Pagaucho 0.49km 25m 5.1%,2:58,9.9km/h,113W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
5,recta merkepaisa 0.54km 8m 0.4%,2:35,12.6km/h,112W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
6,Primavera - 1er Km Minas 3.69km 96m 2.6%,16:59,13.1km/h,188W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
7,EL 61 6.06km 363m 6.0%,43:24,8.4km/h,197W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
8,Virgen Climb 1.28km 62m 4.8%,7:15,10.6km/h,211W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."
9,I-25 Climb 1.13km 81m 7.1%,7:56,8.6km/h,204W,"{'Calories': {'Avg': '2830', 'Max': '2830'}, '..."


In [6]:
df=df.append(sgmt,sort=False).reset_index(drop=True)

In [7]:
df.to_json(db_file)

### Save notebook here

In [8]:
f=open('commit.sh','w')
f.write('git commit -am "{}"\n'.format(
    df.iloc[-1]['Info']['name']+' '+df.iloc[-1]['Info']['date']
))
f.write('git push origin master')
f.close()

In [9]:
!bash commit.sh

[master 97ac5eb] Morning Ride 7:36 AM on Sunday, October 11, 2020
 2 files changed, 46 insertions(+), 34 deletions(-)
 rewrite db/strava.json (69%)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (5/5), 16.88 KiB | 2.81 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:restrepo/strava.git
   0e73225..97ac5eb  master -> master


In [10]:
if not( input('Close browser?').find('n')>-1 ):
    browser.close()

Close browser? 
